In [1]:
#This performs MH algorithm on a fully-observed birth-death process 0->X->0 with birth parameter A and death parameter B.
import pandas as pd
import numpy as np
from scipy.stats import gamma

In [2]:
#import saved data
pd_full_x=pd.read_csv("full.csv")
full_x= pd_full_x.values
pd_full_t=pd.read_csv("full_time.csv")
full_t= pd_full_t.values
pd_full_react_type=pd.read_csv("full_type.csv")
full_react_type= pd_full_react_type.values

In [3]:
#initializations and resizing of data
N=5000        #length of the chain
nt=2000       #number of observations to use 
A=np.zeros(N)
B=np.zeros(N)
A[0]=1        #initialize the Markov Chain A
B[0]=1        #initialize the Markov Chain B
var_A=1       #set proposal variance for A
var_B=0.05    #set proposal variance for B
Ashape=4.5    #set the shape of the Gamma prior for A
Ascale=1      #set the scale of the Gamma prior for A
Bshape=0.25   #set the shape of the Gamma prior for B
Bscale=1      #set the scale of the Gamma prior for B
x=full_x[0:nt]
x=np.reshape(x,(1,len(x)))
t=full_t[0:nt]
t=np.reshape(t,(1,len(t)))
react_type=full_react_type[0:nt]
react_type=np.reshape(react_type,(1,len(react_type)))

In [4]:
def loglikelihood(y,react_type,aa,bb,t):
    like=0
    size=np.shape(x)
    for i in range(1,size[1]):
        if react_type[0,i]==0:
            like +=np.log(aa)
        else:
            like +=np.log(bb*y[0,i-1])
            
    h0_vector=aa+bb*y    
    like += -(np.trapz(h0_vector,x=t))    
    return like

In [5]:
def accept_rate(var,x,react_type,A,B,prop,t,shape,scale1):
    if var=='birth':
        prop_like=loglikelihood(x,react_type,prop,B,t)+np.log(gamma.pdf(prop,a=shape,scale=scale1))
        current_like=loglikelihood(x,react_type,A,B,t)+np.log(gamma.pdf(A,a=shape,scale=scale1))
    else:
        prop_like=loglikelihood(x,react_type,A,prop,t)+np.log(gamma.pdf(prop,a=shape,scale=scale1))
        current_like=loglikelihood(x,react_type,A,B,t)+np.log(gamma.pdf(B,a=shape,scale=scale1))
    rate=np.minimum(1,np.exp(prop_like-current_like))
    return rate

In [8]:
#Perform Gibbs sampling
for i in range(N-1):
    if i % 2 != 0:     #MH for A
        propA=-1
        while propA<0:
            propA=A[i]+np.random.normal(0,var_A,1)
        rate=accept_rate('birth',x,react_type,A[i],B[i],propA,t,Ashape,Ascale)
        if np.random.uniform(0,1,1)<rate:
            A[i+1]=propA
            B[i+1]=B[i]
        else:
            A[i+1]=A[i]
            B[i+1]=B[i]
    else:              #MH for B
        propB=-1
        while propB<0:
            propB=B[i]+np.random.normal(0,var_B,1)
        rate=accept_rate('death',x,react_type,A[i],B[i],propB,t,Bshape,Bscale)
        if np.random.uniform(0,1,1)<rate:
            B[i+1]=propB
            A[i+1]=A[i]
        else:
            B[i+1]=B[i]
            A[i+1]=A[i]
    

In [12]:
#Diagnostics
#thin
#show trajectory
#show histogram
#show autocorelation function
print(B)

[1.         0.25135111 0.25135111 ... 0.20755311 0.20755311 0.20755311]
